In [ ]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import requests
import sncosmo
import json

from utils import get_config

In [ ]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
config = get_config()

username = config['username']
date = '2018-09-26'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
minfilters = 2      # Reported detections in at least this many filters
minrefframes = 1    # Cut away alert information where this is not positive (how can it not be, but ok...)

marshal_saveprogs = 'AMPEL'   # Candidates already in this program will not be scanned. Reg exp string, eg 'AMPEL|RCF'
marshal_savid = 42
logpath = 'logs/'
slackfilepath = config['slackfilepath']

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sncandid_soup.json', 'r') as fp:
    soupids = json.load(fp)

# log. This should be saved and pushed to github. If everyone does this we should have the full scanning
# history for this run
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)



In [ ]:
df_sum = pd.read_csv(os.path.join(slackfilepath, "Summary_%s.csv"%(date)))
df_phot = pd.read_csv(os.path.join(slackfilepath, "Photometry_%s.csv"%(date)))

if minrefframes>0:
    df_phot = df_phot[df_phot["nframesref"]>minrefframes]
    logger.info("Cutting down alert info to subset with nrefframes keyword")

In [ ]:
logger.info("There were %s candidates reported %s"%(df_sum.shape[0],date))

In [ ]:
# Find already classified SNe
i_untyped = (df_sum["T2-classification"]== "None") | (df_sum["T2-classification"].isnull())
df_classified = df_sum[ ~i_untyped]
df_tmp = df_sum[ i_untyped]
logger.info("Out of these %s already have Marshal type and will be skipped."%len(df_classified))


In [ ]:
# Are there any remaining candidates with spectroscopic redshifts above limit
i_highz = ( (df_tmp['T2-NEDz_z']>maxz) | (df_tmp['T2-milliquas_redshift']>maxz)  | (df_tmp['T2-SDSS_spec_z']>maxz) ) 
df_classified = df_tmp[ i_highz]
df_tmp2 = df_tmp[~i_highz]
logger.info("%s have a catalog z larger than %s  and will be skipped."%(np.sum(i_highz),maxz))


In [ ]:
# Which are already saved in the marshal?
i_saved = df_tmp2["T2-programs"].str.contains(marshal_saveprogs, regex=True, na=False, flags=re.IGNORECASE)
df_saved = df_tmp2[i_saved]
df_candidates = df_tmp2[~i_saved]
logger.info("Detected %s already saved candidates and %s new"%(len(df_saved),len(df_candidates)))

In [ ]:
# Examine new candidates, build a list of the onces fulfilling basic criteria
sncandidates = []
for sne in df_candidates.iterrows():
    print( sne[1]["ztf_name"] )
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    if np.min(magpsf)> minpeakmag:
        msg = "%s: Skipping %s, never reached limit %s"%(date,sne[1]["ztf_name"],minpeakmag)
        logger.info(msg)
        print(msg)
        continue
    if len(magpsf)< mindet:
        msg = "%s: Skipping %s, less than %s detections"%(date,sne[1]["ztf_name"],mindet)
        logger.info(msg)
        print(msg)
        continue

    age = np.max(jd)-np.min(jd)
    if age>maxage:
        msg = "%s: Skipping %s, age %s longer than limit %s"%(date,sne[1]["ztf_name"],age,maxage)
        logger.info(msg)
        print(msg)
        continue
        
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    if len(np.unique(phot["fid"]))<minfilters:
        msg = "%s: Skipping %s, only %s filters"%(date,sne[1]["ztf_name"],len(np.unique(phot["fid"])))
        logger.info(msg)
        print(msg)
        continue
        
    sncandidates.append(sne[1]["ztf_name"])
    continue
        
    

print("Left with %s candidates to scan"%(len(sncandidates)))

In [ ]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Quick plot of SN nbr k
    i : {0:undecided, 1:submit, 2:wait 3:bogus}
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = sncandidates[snnbr]
    if i == 'Wait':
        logger.info("Waiting for further obs for SN %s %s"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Submit':
        logger.info("Sending SN %s %s for possible follow-up"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'Garbage':
        logger.info("I never want to see SN %s %s again!"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(sncandidates):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    # Reset    
    snname = sncandidates[snnbr]
    

    phot = df_phot[ df_phot["ztf_name"]==snname ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']


    plt.title(snname + ' ' + str(snnbr+1) + ' out of '+str(len(sncandidates)))
    plt.plot(jd[filters==1]-Time.now().jd,magpsf[filters==1],'go')
    plt.plot(jd[filters==2]-Time.now().jd,magpsf[filters==2],'ro')
    plt.plot(jd[filters==3]-Time.now().jd,magpsf[filters==3],'ko')
    plt.gca().invert_yaxis()
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)
wiggy = widgets.RadioButtons(
    options=['Nothing','Wait', 'Submit', 'Garbage','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [ ]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [ ]:
# This is the all important list of decisions you have made
decisions = {}

In [ ]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

In [ ]:
# Now fun starts for real. Lets download all the candidates and check whether we find all the candidate ids there
# (cause are the only that are useful for saving). This will take a while.
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

In [ ]:
ingestoldcand_start = Time.now()
print("Starting ingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    logger.info("Trying to ingest %s through avroid %s"%(sncandidates[snnbr],avroid))
    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        logger.info('... already a candidate with id %s'%(canddate['candid']))
        continue
    
    pl.ingest_avro(avroid)
ingestoldcand_end = Time.now()
print("Stoppedingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_end))

In [ ]:
# We do this again because it is soo funny
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

In [ ]:
# And we do this again as well. If this worked we should now have all candidates saved
togrowth = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        continue
    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    msg = "Failed to ingest alert %s for SN %s"%(avroid,sncandidates[snnbr])
    logger.info(msg)
    togrowth.append(msg)
    


In [ ]:
# Write an email to Mansi/Growth team with the following content
print(togrowth)
# And when we hear back from them this SN should be saved. I hope they do it for us

In [ ]:
# Ok - this does not work since we have the wrong candidate id...
missing_candidsne = []
# Now it is time to save remaining candidates
savecand_start = Time.now()
print("Starting saving at %s (not really that useful)"%(savecand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    try:
        candid = soupids[sncandidates[snnbr]]
        logger.info("Try to save %s through candid %s"%(sncandidates[snnbr],candid))
        pl.save_source(candid,42)
    except KeyError:
        logger.info("Lacking candid for %s"%(sncandidates[snnbr]))
        missing_candidsne.append(snnbr)


        
savecand_end = Time.now()
print("Stopped saving at %s"%(savecand_end))

In [ ]:
# Check whether we have missing candidate ids
print("Missing candidate id for these sne: %s"%(missing_candidsne))

In [ ]:
# Reload saved sources
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)

In [ ]:
# Loop through them again to check they are all saved
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=False)
    if canddate is None:
        logger.info('SN %s was not saved. Oj oj.'%(sncandidates[snnbr]))
        continue

    logger.info('SN %s was saved'%(sncandidates[snnbr]))


In [ ]:
## Hopefully done. If all of the candidates were saved (no ojojs) you are done with scanning!
# If candidate IDs are still missing you have to try to scrape the webpages for them as below. Have fun!

In [ ]:
# Trying to get candid through the seargant
from ampel.pipeline.t3.sergeant import marshal_functions
tbase = Time.now()
allcandidates = []

In [ ]:
for k in range(2):
    print(k)
    tend = Time(tbase.jd-k,format='jd')
    tstart = Time(tbase.jd-k-1,format='jd')
    ser = marshal_functions.Sergeant(program_name='AMPEL Test',start_date=tstart.iso,end_date=tend.iso)
    ser.cutprogramidx = 42
    cands = ser.list_scan_sources()
    print("Found %s cands"%(len(cands)))
    allcandidates.extend(cands)

In [ ]:
for cand in allcandidates:
    if cand["name"] in soupids.keys():
        continue
    print("new candidate %s"%(cand["name"]))
    soupids[cand["name"]] = cand["candid"]

In [ ]:
# Save the added dict of candidate id
with open('sncandid_soup.json', 'w') as fp:
    json.dump(soupids, fp)

In [ ]:
# cheat for 09 23
decisions = {0: 'Garbage', 1: 'Garbage', 2: 'Garbage', 3: 'Garbage', 4: 'Wait', 5: 'Garbage', 6: 'Submit', 7: 'Submit', 8: 'Garbage', 9: 'Garbage', 10: 'Garbage', 11: 'Garbage', 12: 'Garbage', 13: 'Garbage', 14: 'Garbage', 15: 'Garbage', 16: 'Submit', 17: 'Submit', 18: 'Submit', 19: 'Garbage', 20: 'Garbage', 21: 'Garbage', 22: 'Garbage', 23: 'Garbage', 24: 'Garbage', 25: 'Garbage', 26: 'Garbage', 27: 'Garbage', 28: 'Garbage', 29: 'Garbage', 30: 'Submit', 31: 'Submit', 32: 'Garbage', 33: 'Submit', 34: 'Submit', 35: 'Submit', 36: 'Garbage', 37: 'Submit', 38: 'Garbage', 39: 'Garbage', 40: 'Garbage'}